# Regression as a machine learning tool

The general purpose of multiple regression (the term was first used by Pearson in 1908) is to learn more about the **relationship** between several **independent variables** or **features** and one or multiple **dependent** or **criterion variables**.

For example, a real estate agent might record for each listing the size of the house (in square feet), the number of bedrooms, the average income in the respective neighborhood according to census data, and a subjective rating of appeal of the house. Once this information has been compiled for various houses it would be interesting to see whether and how these measures relate to the price for which a house is sold. For example, you might learn that the number of bedrooms is a **better predictor** of the price for which a house sells in a particular neighborhood than how "pretty" the house is (subjective rating). You may also detect "outliers," that is, houses that should really sell for more, given their location and characteristics. But how do we find the best predictors?

In this notebook, we will cover through several examples the three main concepts of multiple regression. In Chapter 1, we will start by introducing multiple regression models and see how we can determine the "best" model. In Chapter 2, we will discuss the dangers of overfitting that arose in Chapter 1. In Chapter 3, we will see how **ridge regression** can be used to regulate the predicted relationship between the features and the criterion variable and how it can be used to mitigate overfitting. Finally, in Chapter 4, the concept of **lasso regression** is introduced to automatically select the best sparse set of features to predict the criterion variable.

This notebook is based on the online course *Machine Learning: Regression*, offered by the University of Washington through *Coursera*, part of the [*Machine Learning* specialization](http://www.coursera.org/specializations/machine-learning). For this introductory notebook, we will use pre-implemented machine learning algorithms, provided through [*GraphLab Create*](http://turi.com/index.html). You can install *GraphLab Create* using *Anaconda* by following [these instructions](https://turi.com/download/install-graphlab-create.html). However, note that *GraphLab Create* was created using *Python 2.x*. To be able to run the program, you will have to download *Anaconda2*, also if you already downloaded *Anaconda3*. *GraphLab Create* is free on a 1-year, renewable license for educational purposes. You can register for free on [this page](http://turi.com/download/academic.html).

Once you've downloaded *GraphLab Create* and registered for a license, you can fire up *GraphLab Create* using

In [ ]:
import graphlab

## Chapter 1: Multiple linear regression

In this chapter, we will use data on house sales in King County (Greater Seattle Region, Washington, USA) to predict house prices using multiple regression. The goal of this chapter is to explore multiple regression and feature engineering with predefined functions.

For this chapter, you will need to unpack the `kc_house_data.gl.zip` file that is provided with this notebook in the same folder as the notebook. Once unzipped, you can load in the house sales dataset from King Country as follows:

In [ ]:
sales = graphlab.SFrame('kc_house_data.gl/')

### 1.1. Exploring the dataset

Using the print statement below, we'll get an overview of the first ten entries in the dataset.

In [ ]:
print(sales)

As you can see, each house sale is assigned an unique id. In addition, the date of the house sale is listed, as well as the price paid for the house and a series of features, including how many bedrooms and bathrooms are available, how large the lot and the living space is, the number of floors, whether or not the house is at the waterfront and so on.
The data can be further explored using the *GraphLab Create* dashboard. 

In [ ]:
graphlab.canvas.set_target('ipynb')
sales.show(columns=None, view=None, x=None, y=None)

In the remainder of this notebook, we will be interested to **predict** the house price given a (sub)set of features.

### 1.2. An introduction to linear regression

In **regression**, we are interested to predict a (continuous) variable $y$ based on a set of features $\mathbf{x} = [x_1, x_2, ..., x_d]^T$. Once this regression model $f$ is trained, it will provide a prediction $\hat y_j$ for each input $\mathbf{x}_j$:

$$ \hat y_j  = f(\mathbf{x}_j) = f(x_{j1}, x_{j2}, ..., x_{jd}) $$

In **linear regression**, this model $f$ is a linear function of the input variables $\mathbf{x}$. The coefficients of the function are often denoted by the weight vector $\mathbf{w} = [w_0, w_1, w_2, ..., w_d]^T$, and need to be trained before using the model. Note that we added a weight $w_0$ without a corresponding feature $x_0$. This is often done, as it allows us to have a constant offset. This is equivalent to adding $x_{j0} = 1$ as the corresponding feature for each entry $j$. Once the weights are given, the model assigns to each set of features a prediction $\hat y_j$:

$$ \hat y_j = \sum_{i=0}^d w_i x_{ji} = w_0 + \sum_{i=1}^d w_i x_{ji} = \mathbf{w}^T \mathbf{x}_j$$

Before predicting, however, we have to **train a model**. In this training stage, a set of data, called the **training set**, will be used to determine the weights $\mathbf{w}$. This training consists of proposing a set of weights, predicting the prices using this guess, and then comparing these predictions to the true prices to see how well our predictive model is . In **unregulated linear regression**, we want to choose the weights $\mathbf{w}$ such that the mismatch between the predictions $\hat{\mathbf{y}} = [\hat y_1, \hat y_2,..., \hat y_N]^T$ ($N$ being the size of the dataset, *i.e.* the number of entries) and the true prices $\mathbf{y} = [y_1, y_2, ..., y_N]^T$ is minimized.

Depending on how you define "mismatch" and the associated **cost**, the best weights $\mathbf{w}$ will differ. A very popular choice for the **cost function** is to look at the **residual sum of squares** between the predicted and the true values. The **residual** of a predicted value is simply the difference $y_j - \hat y_j$. The residual sum of squares then squares this value, and sums over all predictions:

$$ \text{RSS}(\mathbf w) = \sum_{j=1}^N (y_j - \hat y_j)^2 = \sum_{j=1}^N \left(y_j - \sum_{i=0}^d w_i x_{ji} \right)^2 $$

Note that, by defining the mismatch like this, overpredictions and underpredictions are penalized in the same way (it is a symmetric cost function). This residual sum of squares can also be written as the square of the $L_2$ norm of the difference vector $\mathbf{y} - \hat{\mathbf{y}}$:

$$ \text{RSS}(\mathbf w) = || \mathbf{y} - \hat{\mathbf{y}} ||_2^2 = (\mathbf{y} - \hat{\mathbf{y}})^T (\mathbf{y} - \hat{\mathbf{y}}) = (\mathbf{y} - \mathbf{w}^T \mathbf{X})^T (\mathbf{y} - \mathbf{w}^T \mathbf{X}) $$

**Note**: Row $j$ of the $(N \times d)$ matrix $\mathbf{X}$ corresponds to the set of $d$ features for a given entry $j$. Column $i$ of this matrix corresponds to the $N$ values of feature $\mathbf{x}_i$ for every entry. 

Training a linear regression model can hence be summarized as follows:

1. Make a first guess for the weights $\mathbf{w}$
2. Using this weights, define the cost function (here the RSS)
3. Determine the gradient of the cost function, and update the weights using for instance gradient descent
4. Return to step 2 until the gradient is sufficiently small. If this gradient is sufficiently small, you found a (near-)optimal set of weights $\mathbf{w}$

Finding this optimal set of weights $\mathbf{w}$, which is the so-called **least-squares solution**, completely defines the model, which can then be employed on a testing set.

### 1.3. A first linear regression model

To train our first model, we will first need to define a training set, which is a subset of the total dataset **only** used for learning. The test set (and, if necessary, also the validation set) need to be defined before training the model, and can **never** be used in the training. Rather, the test and validation sets are used as a proxy to determine the error of your model on data it has never encountered before.

In the following code, we will randomly assign 80% of the original data for training. 

In [ ]:
train_data, test_data = sales.random_split(.8, seed=0)

Second, we will need to define which features to train our data on. The data set contains 19 possible features, one column with prices (the prediction) and one column of id's. For this first model, we will train a model based on three features: the size of the living area, the number of bedrooms, and the number of bathrooms (in addition to a constant offset which is added by default in *GraphLab Create*):

In [ ]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']

With *GraphLab*, you can easily train a linear model using the following syntax (don't mind the exact syntax, as we will *GraphLab* only for illustrative purposes):

In [ ]:
example_model = graphlab.linear_regression.create(train_data, target='price', features=example_features, 
                                                  validation_set=None)

From the output above, you see that we trained a model with three features ($d=3$) on 17384 entries ($N=17384$), corresponding to 80% of the original dataset.

Now that we have fitted the model we can extract the regression weights (coefficients) as follows:

In [ ]:
example_weight_summary = example_model.get("coefficients")
print(example_weight_summary)

Once a model is trained, we can use the built-in `.predict()` function to find the predicted values for data we pass. For example, using the example model above, we can predict the results of the training data and print out the first prediction:

In [ ]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0])

### 1.4. Assessing the error

Now that we can make predictions, we can also calculate the RSS of the model. The function below does exactly that:

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    prediction = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - prediction
    # Then square and add them up
    RSS = (residuals**2).sum()
    return(RSS)

Let's take a look at the RSS of our model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print(rss_example_train)

Note that, while the RSS is very useful to determine how to improve upon a model, its magnitude in itself is not so intuitive. Very often, the RSS is divided by the number of samples $N$ to get a size-independent result. Moreover, by taking the square root of this resulting value, we get a result that shares its dimensions with the prediction (here the price in US dollars).

### 1.5. Preprocessing the feature space

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms), we can also consider transformations of existing features. For instance, we can consider the (natural) log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first we're first going to import the *NumPy* library.

In [ ]:
import numpy as np

Next, we will create the following four new features:

* `bedrooms_squared = bedrooms*bedrooms`. Squaring bedrooms will increase the separation between not many bedrooms (*e.g.* 1) and lots of bedrooms (*e.g.* 4). Consequently, this feature will mostly affect houses with many bedrooms.
* `bed_bath_rooms = bedrooms*bathrooms` gives what's called an **interaction feature**. It is large when *both* of them are large.
* `log_sqft_living = log(sqft_living)`. Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* `lat_plus_long = lat + long`. Adding latitude to longitude is totally nonsensical but we will do it anyway (you'll see why).

In [ ]:
# Define the bedrooms_squared feature for training and test data
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

# Define the bedrooms*bathrooms feature for training and test data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

# Define the logarithm of the square feet of living feature for training and test data
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: np.log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: np.log(x))

# Define the sum of latitude and longitude feature for training and test data
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

### 1.6. Training and comparing multiple models

Now we will learn the weights for three (nested) models to predict house prices. The first model will have the fewest features, the second model will add one more feature, and the third will add a few more:
* Model 1: Squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: Add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that we have the features, we can learn the weights for the three different models to predict `target = 'price'` using `graphlab.linear_regression.create()`:

In [ ]:
model_1 = graphlab.linear_regression.create(train_data, target='price', features=model_1_features, validation_set=None)
model_2 = graphlab.linear_regression.create(train_data, target='price', features=model_2_features, validation_set=None)
model_3 = graphlab.linear_regression.create(train_data, target='price', features=model_3_features, validation_set=None)

We can now extract the coefficients of each model:

In [ ]:
model_1_coef = model_1.get('coefficients')
print(model_1_coef)

model_2_coef = model_2.get('coefficients')
print(model_2_coef)

model_3_coef = model_3.get('coefficients')
print(model_3_coef)

Take a look at the sign of the weight for `bathrooms` in model 1 and model 2. What does this mean?

Now that we've trained three models and extracted the model weights we want to evaluate which model is best. First we will compute the RSS on the training data for each of the three models using our previously defined function:

In [ ]:
print('Model 1 training RSS: ' + str(get_residual_sum_of_squares(model_1, train_data, train_data['price'])))
print('Model 2 training RSS: ' + str(get_residual_sum_of_squares(model_2, train_data, train_data['price'])))
print('Model 3 training RSS: ' + str(get_residual_sum_of_squares(model_3, train_data, train_data['price'])))

How does the RSS on the training data evolve as a function of the size of the feature space? Is this what you expect?

Now we will compute the RSS on on test data for each of the three models. Note that the test data was not used to train the model.

In [ ]:
print('Model 1 testing RSS: ' + str(get_residual_sum_of_squares(model_1, test_data, test_data['price'])))
print('Model 2 testing RSS: ' + str(get_residual_sum_of_squares(model_2, test_data, test_data['price'])))
print('Model 3 testing RSS: ' + str(get_residual_sum_of_squares(model_3, test_data, test_data['price'])))

How does the RSS on the test data evolve as a function of the size of the feature space? Is this what you expect? Think about the features that were added to each model from the previous. Does it makes sense why you have to keep the test data separate when training the model?

<br/>

## Chapter 2: The curse of overfitting

The example above, in which the RSS does not decrease with increasing feature space size, indicates that the larger model 3 is overfitted. This means that we assign too much importance to features which do not **generalize** well to the true relation between the features and the prediction. To observe this problem of **overfitting** more prominently, let's take a look at an analytical model. Here, we will try to predict $y = \sin(4x)$ given 30 points to which we added some artificial noise.

### 2.1. Setting the stage

First, we'll have to import some more packages.

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

First, we will create 30 random values for $x$ in the interval $[0,1[$:

In [ ]:
random.seed(98103)
n = 30
x = graphlab.SArray([random.random() for i in range(n)]).sort()

For each of these inputs, we can of course calculate the true outcome:

In [ ]:
y = x.apply(lambda x: np.sin(4*x))

Now, to mimic the noise in real data, we will add random Gaussian noise to $y$:

In [ ]:
random.seed(1)
e = graphlab.SArray([random.gauss(0,1.0/3.0) for i in range(n)])
y = y + e

Finally, to make it easier for us to access and manipulate the data, we will put the data into an SFrame:

In [ ]:
data = graphlab.SFrame({'X1':x,'Y':y})
print(data)

Finally we will define a few functions to make life easier. First, we will create a function to simply plot the data:

In [ ]:
def plot_data(data):    
    plt.plot(data['X1'], data['Y'], 'k.')
    plt.xlabel('x')
    plt.ylabel('y')

plot_data(data)

Second, we would like to expand our features set, as we for the moment only have $x$ as feature (and a constant offset). To do so, we will consider the different powers of $x$ as independent features. For this, we define a function to create our features for a polynomial regression model of any degree:

In [ ]:
def polynomial_features(data, deg):
    data_copy = data.copy()
    for i in range(1,deg):
        data_copy['X'+str(i+1)] = data_copy['X'+str(i)]*data_copy['X1']
    return(data_copy)

Third, we will define a function to fit the polynomial linear regression model of degree `deg` to the data in `data`:

In [ ]:
def polynomial_regression(data, deg):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=0., l1_penalty=0.,
                                              validation_set=None, verbose=False)
    return(model)

Fourth, we'll create a function to plot the data and the corresponding predictions:

In [ ]:
def plot_poly_predictions(data, model):
    plot_data(data)

    # Get the degree of the polynomial
    deg = len(model.coefficients['value']) - 1
    
    # Create 200 points in the x axis and compute the predicted value for each point
    x_pred = graphlab.SFrame({'X1':[i/200.0 for i in range(200)]})
    y_pred = model.predict(polynomial_features(x_pred, deg))
    
    # plot predictions
    plt.plot(x_pred['X1'], y_pred, 'g-', label='degree ' + str(deg) + ' fit')
    plt.legend(loc = 'upper left')
    plt.axis([0, 1, -1.5, 2])

Finally, we will create a function that prints the polynomial coefficients in a nice way:

In [ ]:
def print_coefficients(model):    
    # Get the degree of the polynomial (subtract one for the offset)
    deg = len(model.coefficients['value']) - 1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print('Learned polynomial for degree ' + str(deg) + ':')
    w.reverse()
    print(str(np.poly1d(w)))

### 2.2. Evaluating the effect of the feature space on overfitting

Let's fit a second-order polynomial to the data generated above:

In [ ]:
model_deg2 = polynomial_regression(data, deg=2)

What coefficients did we learn for this second-order model?

In [ ]:
print_coefficients(model_deg2)

Let's predict the thirty outputs with this model, plot them as a continuous line, and compare them to the exact $y$ values with noise:

In [ ]:
plot_poly_predictions(data, model_deg2)

Not too bad! Now, let's do the same for a fourth-order polynomial:

In [ ]:
model_deg4 = polynomial_regression(data, deg=4)
print_coefficients(model_deg4)
plot_poly_predictions(data, model_deg4)

And finally, for a polynomial of order sixteen:

In [ ]:
model_deg16 = polynomial_regression(data, deg=16)
print_coefficients(model_deg16)
plot_poly_predictions(data, model_deg16)

You see that, by increasing the size of the feature space, we succeed in finding a curve that is closer to the output values. However, is this really what we want? Note that, in the end, we're just looking for the sinusoid from which we drew the data in the first place, and that we want to avoid fitting to the noise. If you take a look at the above coefficients for the polynomial of order sixteen, you'll notice that they are extremely large. Moreover, the fit itself seems pretty wild, too, given the simple sinusoid from which we started.

Examples such as the polynomial of order sixteen above show that it is dangerous to only minimize the RSS, without taking into account other factors. Indeed, the polynomial above does not predict the underlying sinusoid well, despite having a lower RSS on the training set (the same was true for our house sales model). In the following two chapters, we will see how **regulation** can minimize **overfitting** to the training set.

<br/>

## Chapter 3: Ridge regression ($L_2$ norm)

As we've seen in the sinusoid example above, overfitting is typically associated with large coefficients. One way to avoid overfitting is hence to penalize large coefficients. In **ridge regression**, this is done by constructing the cost function as the sum of the previously discussed RSS and a term proportional to the square of the $L_2$ norm of the weight vector:

$$\text{cost}(\mathbf{w}) = \text{RSS}(\mathbf{w}) + \lambda ||\mathbf{w}||_2^2 = || \mathbf{y} - \hat{\mathbf{y}} ||_2^2 + \lambda ||\mathbf{w}||_2^2 $$

The strength of this penalty, and thus the fit versus model complexity balance, is controlled by a parameter $\lambda$, which we call the **$L_2$ penalty**.

### 3.1. The sinusoid revisited

Let's take a look at how the fit to the sinusoidal model will change when adding $L_2$ regularization. First, we will define a function to solve the ridge objective for a polynomial regression model of any degree:

In [ ]:
def polynomial_ridge_regression(data, deg, l2_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=l2_penalty,
                                              validation_set=None, verbose=False)
    return(model)

Now, we will perform a ridge fit of a polynomial of order sixteen using a very small penalty:

In [ ]:
model_deg16_l2small = polynomial_ridge_regression(data, deg=16, l2_penalty=1e-25)
print_coefficients(model_deg16_l2small)
plot_poly_predictions(data, model_deg16_l2small)

This function looks very similar to the one we obtained before, without $L_2$ regularization.

Now, let's use a large penalty strength:

In [ ]:
model_deg16_l2large = polynomial_ridge_regression(data, deg=16, l2_penalty=100)
print_coefficients(model_deg16_l2large)
plot_poly_predictions(data, model_deg16_l2large)

We see that this model is less complex, with much smaller coefficients (but also a larger RSS). It seems like we chose our $\lambda$ too large, as this definitely does not resemble a sinusoid.

Let's take a look at fits for a sequence of increasing $\lambda$ values to get an idea what would be the ideal value:

In [ ]:
for l2_penalty in [1e-25, 1e-10, 1e-6, 1e-3, 1e2]:
    model = polynomial_ridge_regression(data, deg=16, l2_penalty=l2_penalty)
    print('lambda = %.2e' % l2_penalty)
    print_coefficients(model)
    print('\n')
    plt.figure()
    plot_poly_predictions(data,model)
    plt.title('Ridge, lambda = %.2e' % l2_penalty)

From the coefficients and the plots above, we see that increasing the $\lambda$ parameter gives smoother, less complex functions, with smaller coefficients. However, this comes at the expense of a larger RSS on the training set. How can we now define a good value for $\lambda$?

Typically, for sufficiently large datasets, one will define not only a training and a test set, but also a validation set. Then, for each value of $\lambda$, a different model will be trained on the training set, and its RSS on the test set will be considered as a proxy for the error of the model. Then, that $\lambda$ value that minimizes the RSS on this test set will be chosen as the best $\lambda$ value. However, as we have now used the test set to choose the best model, we can no longer use it as a proxy for the error of the model. Hence, as a last step, we will need to calculate the RSS on the validation set.

While this is the ideal validation method, it does rely on a sufficiently large dataset. If the dataset is too small, we can use an approximate method called **cross validation**. Without going into too much detail, cross validation considers several separations of the data into training and test data, calculates the RSS on the test data for each trained model, and averages these RSSs to determine the best $\lambda$ parameter.

Here, we'll consider "leave one out" (LOO) cross validation, which one can show approximates average mean square error (MSE). As a result, choosing $\lambda$ to minimize the LOO error is equivalent to choosing $\lambda$ to minimize an approximation to average MSE.

In [ ]:
# LOO cross validation -- return the average MSE
def loo(data, deg, l2_penalty_values):
    # Create polynomial features
    data = polynomial_features(data, deg)
    
    # Create as many folds for cross validatation as number of data points
    num_folds = len(data)
    folds = graphlab.cross_validation.KFold(data,num_folds)
    
    # for each value of l2_penalty, fit a model for each fold and compute average MSE
    l2_penalty_mse = []
    min_mse = None
    best_l2_penalty = None
    for l2_penalty in l2_penalty_values:
        next_mse = 0.0
        for train_set, validation_set in folds:
            # train model
            model = graphlab.linear_regression.create(train_set,target='Y', 
                                                      l2_penalty=l2_penalty,
                                                      validation_set=None, verbose=False)
            
            # predict on validation set 
            y_test_predicted = model.predict(validation_set)
            # compute squared error
            next_mse += ((y_test_predicted-validation_set['Y'])**2).sum()
        
        # save squared error in list of MSE for each l2_penalty
        next_mse = next_mse/num_folds
        l2_penalty_mse.append(next_mse)
        if min_mse is None or next_mse < min_mse:
            min_mse = next_mse
            best_l2_penalty = l2_penalty
            
    return(l2_penalty_mse, best_l2_penalty)

Run LOO cross validation for `num` values of $\lambda$, on a log scale:

In [ ]:
l2_penalty_values = np.logspace(-4, 10, num=10)
l2_penalty_mse, best_l2_penalty = loo(data, 16, l2_penalty_values)

Plot the results of estimating LOO for each value of $\lambda$:

In [ ]:
plt.plot(l2_penalty_values, l2_penalty_mse, 'k-')
plt.xlabel('$\ell_2$ penalty')
plt.ylabel('LOO cross validation error')
plt.xscale('log')
plt.yscale('log')

Now, let's find the value of $\lambda$, $\lambda_{\mathrm{CV}}$, that minimizes the LOO cross validation error:

In [ ]:
best_l2_penalty

Given this best model, we can now fit the best model, and see how well it performs:

In [ ]:
model_bestl2 = polynomial_ridge_regression(data, deg=16, l2_penalty=best_l2_penalty)
print_coefficients(model_bestl2)
plot_poly_predictions(data, model_bestl2)

That doesn't seem bad at all, with a nice balance between minimizing the accuracy and minimizing the complexity of the model. Note: finding this balance is also called the **bias-variance trade-off**.

### 3.2. Overfitting in the house sales data

To observe overfitting in the house sales data, we will define a function `polynomial_sframe` that creates an SFrame with columns containing the powers of a given input:

In [ ]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature.apply(lambda x: x**power)
    return(poly_sframe)

Let's use *Matplotlib* to visualize what a polynomial regression looks like on the house data. We will use the `sqft_living` variable as basis to generate polynomials. For plotting purposes (connecting the dots), we'll need to sort by the values of `sqft_living`. For houses with identical square footage, we break the tie by their prices.

In [ ]:
sales = graphlab.SFrame('kc_house_data.gl/')
sales = sales.sort(['sqft_living','price'])

We will create a polynomial of order fifteen using the `sqft_living` variable as input, and an $L_2$ penalty of $10^{-5}$:

In [ ]:
l2_small_penalty = 1e-5

poly15_data = polynomial_sframe(sales['sqft_living'], 15)
poly15_features = poly15_data.column_names()
poly15_data['price'] = sales['price']
model15_small = graphlab.linear_regression.create(poly15_data, target='price',
                                                  features=poly15_features,
                                                  l2_penalty=l2_small_penalty,
                                                  validation_set=None)
print model15_small.get('coefficients')

Based on what we've seen before, we expect this fit to behave wildly. To see whether this is indeed the case, let's split the data in four subsets of about the same size:

In [ ]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, we will generate a model for each of the four subsets:

In [ ]:
def fit_model_15(data):
    poly15_data = polynomial_sframe(data['sqft_living'],15)
    poly15_features = poly15_data.column_names()
    poly15_data['price'] = data['price']
    return graphlab.linear_regression.create(poly15_data, target='price',
                                                  features=poly15_features,
                                                  l2_penalty=l2_small_penalty,
                                                  validation_set=None)

In [ ]:
model15_set1 = fit_model_15(set_1)
model15_set2 = fit_model_15(set_2)
model15_set3 = fit_model_15(set_3)
model15_set4 = fit_model_15(set_4)

Let's now plot the four subsets and the four fits on top of each other:

In [ ]:
plt.plot(set_1['sqft_living'],set_1['price'],'.',
        set_1['sqft_living'], model15_set1.predict(polynomial_sframe(set_1['sqft_living'],15)),'-')

plt.plot(set_2['sqft_living'],set_2['price'],'.',
        set_2['sqft_living'], model15_set2.predict(polynomial_sframe(set_2['sqft_living'],15)),'-')

plt.plot(set_3['sqft_living'],set_3['price'],'.',
        set_3['sqft_living'], model15_set3.predict(polynomial_sframe(set_3['sqft_living'],15)),'-')

plt.plot(set_4['sqft_living'],set_4['price'],'.',
        set_4['sqft_living'], model15_set4.predict(polynomial_sframe(set_4['sqft_living'],15)),'-')

In [ ]:
print('Value of power_1 for set_1: '+ str(model15_set1.get('coefficients')['value'][1]))
print('Value of power_1 for set_2: '+ str(model15_set2.get('coefficients')['value'][1]))
print('Value of power_1 for set_3: '+ str(model15_set3.get('coefficients')['value'][1]))
print('Value of power_1 for set_4: '+ str(model15_set4.get('coefficients')['value'][1]))

The four curves differ from one another a lot, as do the coefficients you learned. The model has a **high variance**. We will see in a moment that ridge regression reduces such variance.

### 3.3. Ridge regression to the rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be (too) large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because the `sqft_living` input is in the order of thousands.)

With the argument `l2_penalty = 1e5`, we will fit a new 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`:

In [ ]:
l2_large_penalty = 1e5

def fit_model_15_large(data):
    poly15_data = polynomial_sframe(data['sqft_living'],15)
    poly15_features = poly15_data.column_names()
    poly15_data['price'] = data['price']
    return graphlab.linear_regression.create(poly15_data, target='price',
                                                  features=poly15_features,
                                                  l2_penalty=l2_large_penalty,
                                                  validation_set=None)

In [ ]:
model15_set1_large = fit_model_15_large(set_1)
model15_set2_large = fit_model_15_large(set_2)
model15_set3_large = fit_model_15_large(set_3)
model15_set4_large = fit_model_15_large(set_4)

In [ ]:
plt.plot(set_1['sqft_living'],set_1['price'],'.',
        set_1['sqft_living'], model15_set1_large.predict(polynomial_sframe(set_1['sqft_living'],15)),'-')

plt.plot(set_2['sqft_living'],set_2['price'],'.',
        set_2['sqft_living'], model15_set2_large.predict(polynomial_sframe(set_2['sqft_living'],15)),'-')

plt.plot(set_3['sqft_living'],set_3['price'],'.',
        set_3['sqft_living'], model15_set3_large.predict(polynomial_sframe(set_3['sqft_living'],15)),'-')

plt.plot(set_4['sqft_living'],set_4['price'],'.',
        set_4['sqft_living'], model15_set4_large.predict(polynomial_sframe(set_4['sqft_living'],15)),'-')

In [ ]:
print('Value of power_1 for set_1: '+ str(model15_set1_large.get('coefficients')['value'][1]))
print('Value of power_1 for set_2: '+ str(model15_set2_large.get('coefficients')['value'][1]))
print('Value of power_1 for set_3: '+ str(model15_set3_large.get('coefficients')['value'][1]))
print('Value of power_1 for set_4: '+ str(model15_set4_large.get('coefficients')['value'][1]))

These curves vary a lot less, now that we've applied a high degree of regularization. Also take a look at the coefficients for `power_1` and compare to the results without regularization. Much better, no?

### 3.4. Selecting an $L_2$ penalty via cross validation

Just like the polynomial degree, the $L_2$ penalty is a "magic" parameter we need to select. We will implement a kind of cross-validation called **$k$-fold cross-validation**. The method gets its name because it involves dividing the training set into $k$ segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process $k$ times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment $k-1$ as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the $k$ validation errors, and use it as an estimate of the generalization error. Notice that all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. *GraphLab Create* has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder.

In [ ]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)
n = len(train_valid_shuffled)

Without going into detail, the function below performs cross validation as explained above, and returns the error (normalized on the number of segments):

In [ ]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    validation_error = 0
    for i in range(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation_set = data[start:end+1]
        training_set = data[:start].append(data[end+1:])
        model = graphlab.linear_regression.create(training_set, target=output_name,
                                                  features = features_list,
                                                  l2_penalty = l2_penalty,
                                                  validation_set = validation_set,
                                                  verbose = False)
        validation_error += ((validation_set[output_name]-model.predict(validation_set))**2).sum()
    return(validation_error/k)

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. The loop defined below does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [$10^1$, $10^{1.5}$, $10^2$, $10^{2.5}$, ..., $10^7$]
    * Run 10-fold cross-validation with `l2_penalty`
* Report which $L_2$ penalty produced the lowest average validation error

In [ ]:
num_points = 13
validation_errors = np.empty(num_points)
idx = 0

poly_data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
poly_features = poly_data.column_names()
poly_data['price'] = train_valid_shuffled['price']

l2_penalties = np.logspace(1, 7, num=num_points)

for l2_penalty in l2_penalties:
    print('Checking L2 penalty of ' + str(l2_penalty))
    validation_errors[idx] = k_fold_cross_validation(10, l2_penalty, poly_data, 'price', poly_features)
    idx += 1

min_idx = np.argmin(validation_errors)

print('According to 10-fold validation, the lowest validation error is ' + str(validation_errors[min_idx]))
print('This is achieved with a L2 penalty of ' + str(l2_penalties[min_idx]))

You may find it useful to plot the $k$-fold cross-validation errors you have obtained to better understand the behavior of the method.

In [ ]:
plt.plot(np.logspace(1, 7, num=num_points), validation_errors, '-')
plt.xscale('log')
print(validation_errors)

Once we've found the best value for the $L_2$ penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, our final model will be trained on the entire dataset.

In [ ]:
final_model = graphlab.linear_regression.create(poly_data, target='price',
                                                  features=poly_features,
                                                  l2_penalty=1000,
                                                  validation_set=None)

Now, let's take a look at the resulting RSS on this test set:

In [ ]:
print(get_residual_sum_of_squares(final_model, polynomial_sframe(test['sqft_living'], 15), test['price']))

How does this compare to the lowest RSS obtained without regularization?

<br/>

## Chapter 4: Lasso regression ($L_1$ norm)

While ridge regression above succeeds in avoiding overfitting by reducing the coefficients, the resulting coefficients are not exactly zero. When we are interested in reducing the size of the feature space, we want to combine the advantages of ridge regression with the automatic reduction of the feature space.

Luckily, this is exactly where **lasso regression** comes into play. In lasso regression, the cost function is defined as the sum of the previously discussed RSS and a term proportional to the $L_1$ norm of the weight vector (*e.g.*, the sum of the absolute values of its elements):

$$\text{cost}(\mathbf{w}) = \text{RSS}(\mathbf{w}) + \lambda ||\mathbf{w}||_1 = || \mathbf{y} - \hat{\mathbf{y}} ||_2^2 + \lambda ||\mathbf{w}||_1 $$

The strength of this penalty, and thus the fit versus model complexity balance, is once again controlled by the parameter $\lambda$. Here, this parameter is called the **$L_1$ penalty**.

Before studying a few examples, let's think about why this penalty puts some coefficients exactly to zero, in contrast to the $L_2$ penalty. First off, the contours of constant RSS are centered around the value $\mathbf{w}$ that would be obtained without regularization. Now, imagine that $\mathbf{w} = [w_1,w_2]^T$ only contains two elements. The contourlines of constant $L_2$ penalty then form a circle centered about the origin. However, the contourlines of constant $L_1$ penalty form a square with corners located on the axes. When optimizing, we want to minimize the sum of the RSS and the penalty, which can be found as the intersection of the contour lines of the RSS on the one hand and the contour lines of the penalty on the other hand. In the case of the $L_1$ penalty, it is much more likely that this intersection occurs on a given axis, hence putting the other weights to zero.

### 4.1. The sinusoid revisited (part 2)

Let's take a look again at our sinusoid. First, we will define a function to solve the lasso objective for a polynomial regression model of any degree:

In [ ]:
def polynomial_lasso_regression(data, deg, l1_penalty):
    model = graphlab.linear_regression.create(polynomial_features(data,deg), 
                                              target='Y', l2_penalty=0.,
                                              l1_penalty=l1_penalty,
                                              validation_set=None, 
                                              solver='fista', verbose=False,
                                              max_iterations=3000, convergence_threshold=1e-10)
    return(model)

Similar to the exploration with the $L_2$ penalty, let's visualize the results with a series of $L_1$ penalties:

In [ ]:
for l1_penalty in [0.0001, 0.01, 0.1, 10]:
    model = polynomial_lasso_regression(data, deg=16, l1_penalty=l1_penalty)
    print('l1_penalty = %e' % l1_penalty)
    print('number of nonzeros = %d' % (model.coefficients['value']).nnz())
    print_coefficients(model)
    print('\n')
    plt.figure()
    plot_poly_predictions(data, model)
    plt.title('LASSO, lambda = %.2e, # nonzeros = %d' % (l1_penalty, (model.coefficients['value']).nnz()))

We see that, as $\lambda$ increases, we get sparser and sparser solutions. However, even for our nonsparse case  ($\lambda=0.0001$), the fit of our high-order polynomial is not too wild. This is because, like in ridge, coefficients included in the lasso solution are shrunk relative to those of the least squares (unregularized) solution. This leads to better behavior even without sparsity. Of course, as $\lambda$ goes to 0, the amount of this shrinkage decreases and the lasso solution approaches the (wild) least squares solution.

### 4.2. Lasso regression on the house sales database

Let's return to our house sales database, and let's fit a model with all available features, including some of the features we defined earlier:

In [ ]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(np.sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(np.sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [ ]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

As indicated above, we can easily fit a model with a given $L_1$ penalty using *GraphLab Create*:

In [ ]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Note that the iteration above terminated before finding the coefficients that minimize the $L_1$ cost function, in contrast to the models with a $L_2$ cost function or the least-squares solution. This is a direct consequence of the fact that there exists a closed expression for the best coefficients with the $L_2$ cost function, while no such closed expression exists when using the $L_1$ cost function. In the latter case, one has to rely to true optimization.

Now, let's take a look at those features that have nonzero and those that have zero weights:

In [ ]:
coef = model_all.get('coefficients')
print("Coefficients with zero weights:")
print(coef[coef['value']==0.0])
print("\n")
print("Coefficients with nonzero weights:")
print(coef[coef['value']!=0.0])

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. From this, we can deduce that of all possible features, `bathrooms`, `sqft_living`, `sqft_living_sqrt`, `grade`, and `sqft_above` are the most important features.

### 4.3. Selecting an $L_1$ penalty

To find a good $L_1$ penalty, we will explore multiple values using a validation set. Let us do a three-way split into train, validation, and test sets:

In [ ]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we will write a loop that does the following:
* For `l1_penalty` in [$10^1$, $10^{1.5}$, $10^2$, $10^{2.5}$, ..., $10^7$].
    * Fit a regression model with a given `l1_penalty` on train data.
    * Compute the RSS on the validation data
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [ ]:
RSS_all = np.empty(13)
l1_penalty_all = np.logspace(1,7, num=13)
for idx in range(13):
    l1_penalty = l1_penalty_all[idx]
    print('Processing L1 penalty of ' + str(l1_penalty))
    model_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, l2_penalty=0.0,
                                             l1_penalty=l1_penalty, verbose=False)
    predictions = model_l1.predict(validation)
    print(((validation['price']-predictions)**2).sum())
    RSS_all[idx] = ((validation['price']-predictions)**2).sum()

print('\n')
for idx in range(13):
    print(str(l1_penalty_all[idx]) + ': \t' + str(RSS_all[idx]))
    
min_idx = 0
RSS_min = RSS_all[0]

for idx in range(13):
    if RSS_all[idx] < RSS_min:
        min_idx = idx

print('\n')
print('Minimal RSS for L1 penalty of ' + str(l1_penalty_all[min_idx]) + ': RSS = ' + str(RSS_all[min_idx]))

Now, with this best value for the $L_1$ penalty, we can train our model:

In [ ]:
model_10 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set=None, l2_penalty=0.0,
                                             l1_penalty=l1_penalty, verbose=False)

coef_10 = model_10.get('coefficients')
coef_10_nonzero = coef_10[coef_10['value']!=0]
coef_10_zero = coef_10[-coef_10['value']==0]

print('There are ' + str(len(coef_10_nonzero)) + ' nonzero weights')
print(coef_10_nonzero)
print('and ' + str(len(coef_10_zero)) + ' zero weights')
print(coef_10_zero)

Well, that's rather inconvenient...

### 4.4. Limiting the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, seven features? This may be important if we want to derive "a rule of thumb": an interpretable model that has only a few features in them.

In this section, we are going to implement a simple, two phase procedure to achieve this goal:
1. We'll explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of nonzero weights
2. We'll further explore the narrow region we've found to find a good value for tha `l1_penalty` that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for `l1_penalty`

In [ ]:
max_nonzeros = 7

For the first step, let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, we'll implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data.
    * Extract the weights of the model and count the number of nonzeros.

In [ ]:
nonzeros = np.empty(len(l1_penalty_values))

for idx in range(len(l1_penalty_values)):
    l1_penalty = l1_penalty_values[idx]
    print('Processing L1 penalty of ' + str(l1_penalty))
    model_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set = None, l2_penalty=0.0,
                                             l1_penalty=l1_penalty, verbose=False)
    nonzeros[idx] = model_l1['coefficients']['value'].nnz()
    
print(nonzeros)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few nonzeros, and at the other end, we will have an `l1_penalty` that has too many nonzeros.  

More formally, we'll find:
* The largest `l1_penalty` that has more nonzeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many nonzero weights)
* The smallest `l1_penalty` that has fewer nonzeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few nonzero weights)

In [ ]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]

print('Min L1 penalty: ' + str(l1_penalty_min))
print('Max L1 penalty: ' + str(l1_penalty_max))

We'll now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In the following loop, we'll find the model that has the lowest RSS on the validation set *and* has sparsity equal to `max_nonzeros` (which is seven in this case).

In [ ]:
RSS_all = np.empty(len(l1_penalty_values))
nonzeros_all = np.empty(len(l1_penalty_values))

for idx in range(len(l1_penalty_values)):
    l1_penalty = l1_penalty_values[idx]
    print('Processing L1 penalty of ' + str(l1_penalty))
    model_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set=None, l2_penalty=0.0,
                                             l1_penalty=l1_penalty, verbose=False)
    nonzeros_all[idx] = model_l1['coefficients']['value'].nnz()
    predictions = model_l1.predict(validation)
    RSS_all[idx] = ((validation['price']-predictions)**2).sum()

Let's now find the index that satisfies both requirements:

In [ ]:
RSS_min = 1e100
idx_min = -1
for idx in range(len(RSS_all)):
    if nonzeros_all[idx] == max_nonzeros and RSS_all[idx] < RSS_min:
        RSS_min = RSS_all[idx]
        idx_min = idx
        
print('Requirements satisfied for L1 penalty of ' + str(l1_penalty_values[idx_min]))
print('corresponding to index ' + str(idx_min))

To finalize our search, let's look at which seven features are deemed the most important in our model:

In [ ]:
model_11 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             validation_set=None, l2_penalty=0.0,
                                             l1_penalty=l1_penalty_values[11], verbose=False)

coef_11 = model_11.get('coefficients')

print('Zero coefficients:')
print(coef_11[coef_11['value'] == 0.0])
print('\n')
print('Nonzero coefficients:')
print(coef_11[coef_11['value'] != 0.0])

We'll finish this notebook with two remarks:

1. When comparing the nonzero coefficients in this model and in the simple model we determined *ad hoc* in section 4.2, we see that the six nonzero features retained there are a subset of the seven nonzero features retained here. This is not always the case.
2. While, in general, a model trained with an $L_1$ penalty will have a larger RSS than one trained with a $L_2$ penalty and sometimes an RSS that is even larger than a model trained without penalties, only a $L_1$ (out of these three) will exactly put some weights to zero, and can be used for dimension reduction.

<br/>

Congratulations, you finished this notebook on regression!